In [1]:
import myutil as mu
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset  # 텐서데이터셋
from torch.utils.data import DataLoader  # 데이터로더
from torch.utils.data import Dataset
import matplotlib.pyplot as plt  # 맷플롯립사용



--- 
 - 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨) 


In [2]:

torch.manual_seed(1)

z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
mu.log("z", z)
mu.log("hypothesis", hypothesis)
mu.log("hypothesis.sum()", hypothesis.sum())

z = torch.rand(3, 5, requires_grad=True)
mu.log("z", z)
hypothesis = F.softmax(z, dim=1)
mu.log("hypothesis", hypothesis)
mu.log("hypothesis.sum(dim=1)", hypothesis.sum(dim=1))

y = torch.randint(5, (3,))
mu.log("y", y)
hypothesis = F.softmax(z, dim=1)
mu.log("hypothesis", hypothesis)
y_one_hot = torch.zeros_like(hypothesis)
mu.log("y_one_hot", y_one_hot)
mu.log("y.unsqueeze(1)", y.unsqueeze(1))
mu.log("y_one_hot.scatter_(dim=1, y.unsqueeze(dim=1), index=1)", y_one_hot.scatter_(1, y.unsqueeze(1), 1))




z : 
    torch.Size([3]) tensor([1., 2., 3.])

hypothesis : 
    torch.Size([3]) tensor([0.0900, 0.2447, 0.6652])

hypothesis.sum() : 
    torch.Size([]) 1.0

z : 
    torch.Size([3, 5]) tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
            [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
            [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]])

hypothesis : 
    torch.Size([3, 5]) tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
            [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
            [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]])

hypothesis.sum(dim=1) : 
    torch.Size([3]) tensor([1.0000, 1.0000, 1.0000])

y : 
    torch.Size([3]) tensor([0, 2, 1])

hypothesis : 
    torch.Size([3, 5]) tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
            [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
            [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]])

y_one_hot : 
    torch.Size([3, 5]) tensor([[0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.]])


--- 
 - 이제 비용 함수 연산을 위한 재료들을 전부 손질했습니다. 
 - 소프트맥스 회귀의 비용 함수는 다음과 같았습니다. 
 ![](https://render.githubusercontent.com/render/math?math=cost(W)%20=%20-\frac{1}{n}%20\sum_{i=1}^{n}%20\sum_{j=1}^{k}y_{j}^{(i)}\%20log(p_{j}^{(i)})) 


In [3]:

cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
mu.log("cost", cost)



cost : 
    torch.Size([]) 1.468920350074768



--- 
 - 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨) 
 - 이제 소프트맥스의 비용 함수를 좀 더 하이-레벨로 구현하는 방법에 대해서 알아봅시다. 


In [4]:

z = torch.rand(3, 5, requires_grad=True)
mu.log("z", z)
y = torch.randint(5, (3,))
mu.log("y", y)
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)
mu.log("y_one_hot", y_one_hot)
hypothesis = F.softmax(z, dim=1)
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
mu.log("cost low level", cost)



z : 
    torch.Size([3, 5]) tensor([[0.9371, 0.6556, 0.3138, 0.1980, 0.4162],
            [0.2843, 0.3398, 0.5239, 0.7981, 0.7718],
            [0.0112, 0.8100, 0.6397, 0.9743, 0.8300]])

y : 
    torch.Size([3]) tensor([3, 2, 3])

y_one_hot : 
    torch.Size([3, 5]) tensor([[0., 0., 0., 1., 0.],
            [0., 0., 1., 0., 0.],
            [0., 0., 0., 1., 0.]])

cost low level : 
    torch.Size([]) 1.6471147537231445



--- 
 - F.softmax() + torch.log() = F.log_softmax() 


In [5]:

cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
mu.log("cost log_softmax", cost)



cost log_softmax : 
    torch.Size([]) 1.6471147537231445



--- 
 - 여기서 nll이란 Negative Log Likelihood의 약자입니다. 


In [6]:

cost = F.nll_loss(F.log_softmax(z, dim=1), y)
mu.log("cost nll_loss log_softmax", cost)



cost nll_loss log_softmax : 
    torch.Size([]) 1.6471147537231445



--- 
 - F.log_softmax() + F.nll_loss() = F.cross_entropy() 


In [7]:

cost = F.cross_entropy(z, y)
mu.log("cost cross_entropy", cost)


cost cross_entropy : 
    torch.Size([]) 1.6471147537231445

